In [22]:
import numpy as np
from scipy.stats import norm

In [10]:
np.floor(0.51)

np.float64(0.0)

In [13]:
5 ** 5

3125

In [68]:
mean = 19
range_factoor = mean * 0.1
range_factoor
std_approx = (range_factoor * 2) / 6
norm_distr_points = std_approx
percentiles = np.linspace(1e-5, 1-1e-5, 12, endpoint=True)
percentiles = np.linspace(0,1,10,)[1:-1]
print(percentiles)
data_points = np.append(norm.ppf(percentiles, loc=mean, scale=std_approx), [mean - range_factoor, mean + range_factoor])
print(data_points)
list(set([int(np.floor(point)) if point <= mean else int(np.ceil(point)) for point in data_points] + [mean]))




[0.11111111 0.22222222 0.33333333 0.44444444 0.55555556 0.66666667
 0.77777778 0.88888889]
[18.22692778 18.51568387 18.72720604 18.91151681 19.08848319 19.27279396
 19.48431613 19.77307222 17.1        20.9       ]


[17, 18, 19, 20, 21]